In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os, sys
from pathlib import Path


def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")


src_path = find_src_folder()
sys.path.append(str(src_path))

In [3]:
from typing import Dict, Any
from langgraph.graph import START, END, StateGraph
from agents.dungeon.dungeon_state import SuperDungeonState, DungeonEventState
from agents.dungeon.event.dungeon_event_agent import graph_builder as event_graph
from agents.dungeon.monster.monster_agent_with_llm import MonsterCompositionAgent
from enums.LLM import LLM


# 이벤트 노드
def event_node(state: SuperDungeonState) -> Dict[str, Any]:
    """
    Event Agent를 실행하는 노드
    - 히로인 정보와 던전 정보를 기반으로 이벤트 생성
    """
    print("\n[Event Node] 이벤트 생성 시작...")

    heroine_data = state.get("heroine_data", {}) or {}
    dungeon_base = state.get("dungeon_base_data", {}) or {}
    heroine_memories = state.get("heroine_memories", []) or []

    # Event Agent 입력 State 구성
    event_input_state = {
        "heroine_data": heroine_data,
        "heroine_memories": heroine_memories,
        "event_room": dungeon_base.get("event_room", 3),
        "next_floor": dungeon_base.get("floor", 2),
    }

    # Event Graph 실행
    event_result = event_graph.compile().invoke(event_input_state)

    print(
        f"[Event Node] 완료: 메인 이벤트 ={event_result.get('selected_main_event', 'N/A')}"
    )

    return {
        "main_event_result": event_result.get("selected_main_event", ""),
        "sub_event_result": event_result.get("sub_event", ""),
    }


# 몬스터 노드 (리팩토링 필요함)
def monster_node(state: SuperDungeonState) -> Dict[str, Any]:
    """
    Monster Agent를 실행하는 노드
    - 던전 맵에 몬스터를 배치하고 밸런싱
    """
    print("\n[Monster Node] 몬스터 밸런싱 시작...")

    # Monster Agent 초기화
    heroine_stats = state.get("heroine_stats", [])
    monster_db = state.get("monster_db", {})
    dungeon_base = state.get("dungeon_base_data", {})

    # 디버깅: 입력 데이터 확인
    print(f"[DEBUG] heroine_stats 개수: {len(heroine_stats)}")
    print(f"[DEBUG] monster_db 개수: {len(monster_db)}")
    print(f"[DEBUG] dungeon_base rooms: {len(dungeon_base.get('rooms', []))}개")
    
    # 각 방의 타입 확인
    for idx, room in enumerate(dungeon_base.get('rooms', [])):
        print(f"[DEBUG] Room {idx}: type={room.get('type')}, size={room.get('size')}, monsters={len(room.get('monsters', []))}")

    # MonsterCompositionAgent 생성 및 실행
    agent = MonsterCompositionAgent(
        heroine_stats=heroine_stats, monster_db=monster_db, model_name=LLM.GPT4_1_MINI
    )

    # 던전 데이터로 몬스터 배치
    filled_dungeon, difficulty_log = agent.process_dungeon(dungeon_base)

    print(f"[Monster Node] 완료: {difficulty_log.get('total_monsters', 0)}마리 배치")

    return {"monster_result": filled_dungeon, "difficulty_log": difficulty_log}


# 병합 노드
def merge_results_node(state: SuperDungeonState) -> Dict[str, Any]:
    """
    Event와 Monster 결과를 알고리즘적으로 병합
    LLM 사용 없이 순수 로직으로 구현
    """
    print("\n[Merge Node] 결과 병합 시작 (알고리즘)...")

    # 1. 기본 던전 데이터 가져오기
    base_data = state.get("dungeon_base_data", {})
    main_event = state.get("main_event_result", "")
    sub_event = state.get("sub_event_result", "")
    monster_result = state.get("monster_result", {})
    difficulty_log = state.get("difficulty_log", {})

    # 2. 최종 JSON 구조 생성
    final_json = {
        # 기본 던전 구조 유지
        "dungeon_id": base_data.get("dungeon_id", "unknown"),
        "floor": base_data.get("floor", 1),
        "rooms": monster_result.get("rooms", base_data.get("rooms", [])),
        # Event 정보 병합
        "events": {
            "main_event": main_event,
            "sub_event": sub_event,
            "event_room_index": base_data.get("event_room", -1),
        },
        # Monster 정보 병합
        "monsters": {
            "total_count": difficulty_log.get("total_monsters", 0),
            "total_difficulty": difficulty_log.get("target_score", 0),
            "difficulty_multiplier": difficulty_log.get("ai_multiplier", 1.0),
        },
        # 메타 정보
        "meta": {
            "ai_reasoning": difficulty_log.get("ai_reasoning", ""),
            "model_used": difficulty_log.get("model_used", "unknown"),
        },
    }

    print(f"[Merge Node] 병합 완료:")
    print(f"  - Main Event: {main_event[:50]}...")
    print(f"  - Monsters: {final_json['monsters']['total_count']}마리")
    print(f"  - Difficulty: x{final_json['monsters']['difficulty_multiplier']}")

    return {"final_dungeon_json": final_json}


# ===== Graph Construction =====
graph_builder = StateGraph(SuperDungeonState)

# 노드 추가
graph_builder.add_node("event_node", event_node)
graph_builder.add_node("monster_node", monster_node)
graph_builder.add_node("merge_results_node", merge_results_node)

# Edge 연결 (병렬 실행)
graph_builder.add_edge(START, "event_node")
graph_builder.add_edge(START, "monster_node")

# 두 노드가 완료되면 merge로 이동
graph_builder.add_edge("event_node", "merge_results_node")
graph_builder.add_edge("monster_node", "merge_results_node")

# 최종 종료
graph_builder.add_edge("merge_results_node", END)

graph_builder

In [4]:
# ===== 테스트 실행 (MockFactory 사용) =====
from core.game_dto.z_muck_factory import MockFactory
from agents.dungeon.monster.models import MonsterMetadata, StatData as MonsterStatData

# Super Dungeon Graph 컴파일
super_graph = graph_builder.compile()

# 1. MockFactory로 던전 기본 데이터 생성
mock_dungeon = MockFactory.create_dungeon_data(player_count=1, room_count=5)
mock_player = MockFactory.create_dungeon_player(player_id=1)

# 2. 히로인 스탯 데이터 (Monster Agent용)
heroine_stats = [
    MonsterStatData(
        hp=mock_player.stats.hp,
        strength=mock_player.stats.strength,
        dexterity=mock_player.stats.dexterity,
        defense=15,  # 기본값
        luck=10,  # 기본값
    )
]

# 3. 몬스터 DB 생성 (간단한 목업)
monster_db = {
    1: MonsterMetadata(
        monster_id=1, hp=100, speed=50, attack=15, attack_speed=1.0, name="슬라임"
    ),
    2: MonsterMetadata(
        monster_id=2, hp=150, speed=60, attack=20, attack_speed=1.2, name="고블린"
    ),
    3: MonsterMetadata(
        monster_id=3, hp=200, speed=70, attack=25, attack_speed=1.5, name="오크"
    ),
    4: MonsterMetadata(
        monster_id=4, hp=250, speed=80, attack=30, attack_speed=1.8, name="트롤"
    ),
    5: MonsterMetadata(
        monster_id=5, hp=500, speed=50, attack=50, attack_speed=2.0, name="보스 몬스터"
    ),
}

# 4. 테스트용 State 구성
# MockFactory의 rooms를 변환하되, monsters를 비워둠 (Monster Agent가 채울 수 있도록)
rooms_data = []
for idx, room in enumerate(mock_dungeon.rooms):
    room_dict = room.model_dump()
    # Monster Agent가 배치할 수 있도록 monsters 비우기
    room_dict["monsters"] = []
    
    # 방 타입 조정 (명확한 구조)
    if idx == 0:
        room_dict["type"] = 0  # 시작방
    elif idx == len(mock_dungeon.rooms) - 1:
        room_dict["type"] = 3  # 보스방
    elif idx == 3:
        room_dict["type"] = 2  # 이벤트방
    else:
        room_dict["type"] = 1  # 전투방
    
    rooms_data.append(room_dict)

test_state = {
    "dungeon_base_data": {
        "dungeon_id": "mock_dungeon_001",
        "floor": 2,
        "event_room": 3,
        "rooms": rooms_data,
    },
    "heroine_data": {
        "heroine_id": mock_player.heroineId,
        "name": f"히로인#{mock_player.heroineId}",
        "memory_progress": mock_player.scenarioLevel * 10,
        "affection": mock_player.affection,
    },
    "heroine_stats": heroine_stats,
    "heroine_memories": [],  # Event Agent에서 DB 조회로 채워질 예정
    "monster_db": monster_db,
    "main_event_result": "",
    "sub_event_result": "",
    "monster_result": {},
    "difficulty_log": {},
    "final_dungeon_json": {},
}

print("=" * 70)
print("🏰 Super Dungeon Agent 테스트 시작 (MockFactory 데이터 사용)")
print("=" * 70)
print(f"📊 생성된 던전 정보:")
print(f"  - 방 개수: {len(rooms_data)}개")
print(f"  - 히로인 ID: {mock_player.heroineId}")
print(f"  - 히로인 HP: {mock_player.stats.hp}")
print(f"  - 히로인 전투력: {heroine_stats[0].combat_score:.1f}")
print(f"  - 몬스터 DB: {len(monster_db)}종류")
print(f"\n📋 방 구조:")
for idx, room in enumerate(rooms_data):
    room_type_name = ["시작방", "전투방", "이벤트방", "보스방"][room["type"]]
    print(f"  - Room {idx}: {room_type_name} (type={room['type']}, size={room['size']})")
print("=" * 70)

# Graph 실행
result = super_graph.invoke(test_state)

print("\n" + "=" * 70)
print("✅ 최종 결과")
print("=" * 70)

🏰 Super Dungeon Agent 테스트 시작 (MockFactory 데이터 사용)
📊 생성된 던전 정보:
  - 방 개수: 5개
  - 히로인 ID: 2
  - 히로인 HP: 430
  - 히로인 전투력: 102.9
  - 몬스터 DB: 5종류

📋 방 구조:
  - Room 0: 시작방 (type=0, size=12)
  - Room 1: 전투방 (type=1, size=14)
  - Room 2: 전투방 (type=1, size=11)
  - Room 3: 이벤트방 (type=2, size=9)
  - Room 4: 보스방 (type=3, size=6)

[Event Node] 이벤트 생성 시작...

[Monster Node] 몬스터 밸런싱 시작...
[DEBUG] heroine_stats 개수: 1
[DEBUG] monster_db 개수: 5
[DEBUG] dungeon_base rooms: 5개
[DEBUG] Room 0: type=0, size=12, monsters=0
[DEBUG] Room 1: type=1, size=14, monsters=0
[DEBUG] Room 2: type=1, size=11, monsters=0
[DEBUG] Room 3: type=2, size=9, monsters=0
[DEBUG] Room 4: type=3, size=6, monsters=0
[Monster Node] 완료: 27마리 배치
[Monster Node] 완료: 27마리 배치
sibal='히로인 데이터와 능력 사용, 현재 상황에 들어갈 구체적인 값({heroine_data}, {heroine_memories}, {selected_main_event}{current_floor})가 아직 실제 데이터로 치환되지 않은 상태입니다.\n\n이야기 전개나 이벤트 생성을 진행하려면 다음 정보가 필요합니다:\n1) <역할> : 제가 어떤 포지션(예: 세계관 설명자, 게임 마스터, 시나리오 작가, 시스템 설계자 등)으로 행동해야 하는지\n2) <목표> : 이번 상

In [5]:
# 결과 출력
import json

final_json = result.get("final_dungeon_json", {})

print("\n📊 Dungeon 기본 정보:")
print(f"  - Dungeon ID: {final_json.get('dungeon_id')}")
print(f"  - Floor: {final_json.get('floor')}")
print(f"  - Rooms: {len(final_json.get('rooms', []))}개")

print("\n🎭 Event 정보:")
events = final_json.get("events", {})
print(f"  - Main Event: {events.get('main_event', 'N/A')[:60]}...")
print(f"  - Sub Event: {events.get('sub_event', 'N/A')[:60]}...")
print(f"  - Event Room: #{events.get('event_room_index', -1)}")

print("\n👹 Monster 정보:")
monsters = final_json.get("monsters", {})
print(f"  - Total Count: {monsters.get('total_count', 0)}마리")
print(f"  - Difficulty: x{monsters.get('difficulty_multiplier', 1.0)}")
print(f"  - Target Score: {monsters.get('total_difficulty', 0)}")

print("\n🔧 Meta 정보:")
meta = final_json.get("meta", {})
print(f"  - Model: {meta.get('model_used', 'unknown')}")
print(f"  - Reasoning: {meta.get('ai_reasoning', 'N/A')[:80]}...")

print("\n" + "=" * 70)
print("📄 전체 JSON 출력:")
print("=" * 70)
print(json.dumps(final_json, indent=2, ensure_ascii=False))


📊 Dungeon 기본 정보:
  - Dungeon ID: mock_dungeon_001
  - Floor: 2
  - Rooms: 5개

🎭 Event 정보:
  - Main Event: asd...
  - Sub Event: 히로인 데이터와 능력 사용, 현재 상황에 들어갈 구체적인 값({heroine_data}, {heroine_m...
  - Event Room: #3

👹 Monster 정보:
  - Total Count: 27마리
  - Difficulty: x1.0
  - Target Score: 102.9

🔧 Meta 정보:
  - Model: gpt-4.1-mini-2025-04-14
  - Reasoning: 히로인의 현재 상태가 평균적이며 특별한 위기나 강력함이 감지되지 않습니다. 기억의 파편에 민감한 몬스터와 적응형 전술을 사용하는 매복형 몬스터를...

📄 전체 JSON 출력:
{
  "dungeon_id": "mock_dungeon_001",
  "floor": 2,
  "rooms": [
    {
      "roomId": 1,
      "type": 0,
      "size": 12,
      "monsters": null
    },
    {
      "roomId": 2,
      "type": 1,
      "size": 14,
      "monsters": [
        {
          "monsterId": 4,
          "posX": 0.83,
          "posY": 0.32
        },
        {
          "monsterId": 4,
          "posX": 0.63,
          "posY": 0.19
        },
        {
          "monsterId": 4,
          "posX": 0.04,
          "posY": 0.1
        },
        {
          "monsterI